In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
# Reload the dataset
data = pd.read_csv("Cleaned_Engineering_books_data.csv", encoding ="latin-1")


In [3]:
data

,title,author,image,desc,download_link,pages,publisher,year,language,file,lower title,title length,large image,medium image,desc_length,clean desc,recommend char,recommend char2
0,Foundation Engineering: Design and Constructio...,B.B.K. Huat,https://covers.zlibcdn2.com/covers100/books/1b...,\n Residual soils are found...,https://1lib.in/dl/1247565/f9e712,256.0,Taylor & Francis,2006.0,english,"PDF, 86.52 MB",foundation engineering: design and constructio...,144,https://covers.zlibcdn2.com/covers300/books/1b...,https://covers.zlibcdn2.com/covers200/books/1b...,1337,Residual soils are found in many parts of the ...,foundation engineering design construction tro...,foundation engineering design construction tro...
1,Principles of Thermodynamics,Jean-Philippe Ansermet,https://covers.zlibcdn2.com/covers100/books/29...,\n In this introductory tex...,https://1lib.in/dl/3685253/9f5b4e,529.0,Cambridge University Press,2019.0,english,"PDF, 11.50 MB",principles of thermodynamics,28,https://covers.zlibcdn2.com/covers300/books/29...,https://covers.zlibcdn2.com/covers200/books/29...,1018,"In this introductory textbook, thermodynamics ...",principle thermodynamics introductory thermody...,principle thermodynamics
2,Thermodynamics: An Engineering Approach 8th Ed...,Yunus A. ÃÂengel,https://covers.zlibcdn2.com/covers100/books/6a...,"\nThermodynamics, An Engineering Approach, eig...",https://1lib.in/dl/2577672/7b237e,1115.0,McGraw-Hill Education,2014.0,english,"PDF, 64.77 MB",thermodynamics: an engineering approach 8th ed...,51,https://covers.zlibcdn2.com/covers300/books/6a...,https://covers.zlibcdn2.com/covers200/books/6a...,1195,"Thermodynamics, An Engineering Approach, eight...",thermodynamics engineering approach 8th thermo...,thermodynamics engineering approach 8th
3,Thermal Physics: Thermodynamics and Statistica...,Robert F. Sekerka,https://covers.zlibcdn2.com/covers100/books/d0...,\nIn Thermal Physics: Thermodynamics and Stati...,https://1lib.in/dl/2717455/9a083e,610.0,Elsevier,2015.0,english,"PDF, 7.57 MB",thermal physics: thermodynamics and statistica...,86,https://covers.zlibcdn2.com/covers300/books/d0...,https://covers.zlibcdn2.com/covers200/books/d0...,1578,In Thermal Physics: Thermodynamics and Statist...,thermal physic thermodynamics statistical mech...,thermal physic thermodynamics statistical mech...
4,"Fundamentals of Physics: Mechanics, Relativity...",R. Shankar,https://covers.zlibcdn2.com/covers100/books/15...,"\n Professor R. Shankar, a ...",https://1lib.in/dl/2577615/46ed97,464.0,Yale University Press,2014.0,english,"PDF, 3.80 MB","fundamentals of physics: mechanics, relativity...",66,https://covers.zlibcdn2.com/covers300/books/15...,https://covers.zlibcdn2.com/covers200/books/15...,1105,"Professor R. Shankar, a well-known physicist a...",fundamental physic mechanic relativity thermod...,fundamental physic mechanic relativity thermod...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,Semantic Knowledge Representation for Informat...,Winfried GÃÂ¶dert,https://covers.zlibcdn2.com/covers100/books/58...,\nThis book covers the basics of semantic web ...,https://1lib.in/dl/3419364/81b614,294.0,De Gruyter Saur,2014.0,english,"PDF, 7.75 MB",semantic knowledge representation for informat...,59,https://covers.zlibcdn2.com/covers300/books/58...,https://covers.zlibcdn2.com/covers200/books/58...,363,This book covers the basics of semantic web te...,semantic knowledge representation information ...,semantic knowledge representation information ...
1196,Information Retrieval Systems: Theory and Impl...,Gerald J. Kowalski,https://covers.zlibcdn2.com/covers100/books/4f...,\n The growth of the Intern...,https://1lib.in/dl/1014014/bbb19e,290.0,The Information Retrieval Series 1,1997.0,english,"PDF, 11.47 MB",information retrieval systems: theory and impl...,56,https://covers.zlibcdn2.com/covers300/books/4f...,https://covers.zlibcdn2.com/covers200/books/4f...,1654,The growth of the Internet and the availabili

In [4]:
# Step 1: Transform the text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')  # Use English stopwords
tfidf_matrix = tfidf_vectorizer.fit_transform(data['recommend char'])

# Step 2: Compute similarity matrix using Cosine Similarity
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [5]:
def get_recommendations(title, cosine_sim_matrix, data, top_n=10):
    """
    Generate a list of top N recommendations based on a given title.
    
    Args:
    - title (str): Title of the book to base recommendations on.
    - cosine_sim_matrix (ndarray): Precomputed cosine similarity matrix.
    - data (DataFrame): The original dataset containing item information.
    - top_n (int): Number of recommendations to return.
    
    Returns:
    - List of recommended items with their similarity scores.
    """
    # Get the index of the item matching the title
    idx = data[data['title'] == title].index[0]
    
    # Get pairwise similarity scores for this item
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    
    # Sort the items by similarity scores (highest first)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the indices of the top_n most similar items (excluding the input item)
    sim_scores = sim_scores[1:top_n+1]
    item_indices = [i[0] for i in sim_scores]
    
    # Return the titles and similarity scores
    return data.iloc[item_indices][['title', 'author']], [i[1] for i in sim_scores]

In [6]:
# Example: Get recommendations for a specific book title
target_title = "Fundamentals of Physics: Mechanics, Relativity, and Thermodynamics"  # Replace with a title from your dataset
recommended_items, similarity_scores = get_recommendations(target_title, cosine_sim_matrix, data)

# Display the recommendations
print(f"Recommendations for '{target_title}':")
for i, (item, score) in enumerate(zip(recommended_items.values, similarity_scores)):
    print(f"{i+1}. {item[0]} by {item[1]} (Similarity: {score:.2f})")

Recommendations for 'Fundamentals of Physics: Mechanics, Relativity, and Thermodynamics':
1. Principles of Physics: From Quantum Field Theory to Classical Mechanics by Jun Ni (Similarity: 0.30)
2. Physics for Scientists and Engineers with Modern Physics by Raymond A. Serway (Similarity: 0.28)
3. Physics for scientists and engineers : an interactive approach by Hawkes (Similarity: 0.26)
4. An Introduction to Thermal Physics by Daniel V. Schroeder (Similarity: 0.20)
5. Theoretical Physics 5: Thermodynamics by Wolfgang Nolting (auth.) (Similarity: 0.17)
6. An Introduction to Quantum Physics: A First Course for Physicists, Chemists, Materials Scientists, and Engineers by Stefanos Trachanas (Similarity: 0.15)
7. Introduction to Statistical Physics, Second Edition by Kerson Huang (Similarity: 0.15)
8. Student solutions manual for Mathematical methods for physics and engineering by K. F. Riley (Similarity: 0.14)
9. Applied Mechanics of Solids by Allan F. Bower (Similarity: 0.14)
10. Electroma

In [7]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Varsayımsal gerçek ve tahmin edilen puanlar
y_true = [4.5, 3.0, 5.0, 4.0, 3.5]  # Gerçek puanlar
y_pred = [4.6, 3.2, 4.8, 4.1, 3.4]  # Tahmin edilen puanlar

# MAE ve RMSE hesaplama
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")


MAE: 0.14
RMSE: 0.15


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import ndcg_score

# TF-IDF modelini oluştur
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(data['recommend char'])

# Örnek gerçek ve tahmin edilen sıralama skorları
true_relevance = np.asarray([[3, 2, 3, 0, 1]])
predicted_relevance = np.asarray([[0.5, 0.2, 0.8, 0.4, 0.6]])

# nDCG hesaplama
ndcg = ndcg_score(true_relevance, predicted_relevance)
print(f"nDCG: {ndcg:.2f}")


nDCG: 0.93


In [56]:
recommendations = [1, 3, 5, 7, 9]  # Sistem önerileri
relevant_items = [3, 5, 9]  # Alakalı öğeler

# Reciprocal Rank Hesaplama
ranks = [1 / (recommendations.index(item) + 1) for item in relevant_items if item in recommendations]
arhr = sum(ranks) / len(recommendations)
print(f"ARHR: {arhr:.2f}")


ARHR: 0.21


In [9]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Varsayımsal öneri ve gerçek etiketler
y_true = [1, 0, 1, 1, 0, 1, 0]  # Gerçek: 1 => Alakalı, 0 => Alakasız
y_pred = [1, 0, 1, 0, 0, 1, 1]  # Tahmin

# Hesaplamalar
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")


Precision: 0.75
Recall: 0.75
F1-Score: 0.75


In [38]:
# Kullanıcı tercihi ve öneriler
preferred_items = ["Web Information Retrieval"]  # Kullanıcı tercihi
recommended_items = data['title'].sample(5).tolist()  # Örnek öneriler

# CHR hesaplama
hits = len(set(recommended_items) & set(preferred_items))
chr = hits / len(preferred_items) * 100
print(f"CHR: {chr:.2f}%")


CHR: 0.00%


In [52]:
# Kullanıcı tercihlerine biraz daha ağırlık verin
# Kullanıcı tercihlerine biraz daha ağırlık verin
recommended_items = (
    data[data['title'].str.contains('Physics', case=False)]['title']
    .sample(3)  # Kullanıcı ilgi alanından 3 öneri
    .tolist()
    + data.sample(2)['title'].tolist()  # Genel çeşitlilik için 2 rastgele öneri
)

# Önerilen kitapları yazdırın
print("Recommended Items:", recommended_items)


# Serendipity ve Novelty'yi yeniden hesaplayın
unexpected_recommendations = [item for item in recommended_items if item not in user_preferred_items]
serendipity = len(unexpected_recommendations) / len(recommended_items)

# Novelty için popülerlik skoru yeniden dengelenebilir
item_popularity = {item: random.randint(1, 100) for item in recommended_items}
novelty_score = sum(1 / item_popularity[item] for item in recommended_items) / len(recommended_items)

print(f"Serendipity: {serendipity:.2f}")
print(f"Novelty: {novelty_score:.2f}")


Recommended Items: ['Fundamentals of statistical and thermal physics', 'Nonlinear Dynamics and Chaos: With Applications to Physics, Biology, Chemistry, and Engineering', 'Physics for Scientists and Engineers with Modern Physics', 'Questionnaire Design: How to Plan, Structure and Write Survey Material for Effective Market Research ', 'Fluid Machinery and Fluid Mechanics: 4th International Symposium (4th ISFMFE)']
Serendipity: 1.00
Novelty: 0.04


In [11]:
# TF-IDF benzerlik matrisinden çeşitlilik hesaplama
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix)
diversity_score = 1 - np.mean(cosine_sim[np.triu_indices_from(cosine_sim, k=1)])
print(f"Diversity: {diversity_score:.2f}")


Diversity: 0.97


In [12]:
# Önerilen ve tüm ürünlerin oranı
all_items = set(data['title'])
recommended_items = set(data['title'].sample(5))  # Örnek öneriler
coverage = len(recommended_items) / len(all_items) * 100
print(f"Coverage: {coverage:.2f}%")


Coverage: 0.42%


In [13]:
from datetime import datetime

# Örneğin, 2015'ten önceki ürünleri eski kabul edelim
cutoff_year = 2015
old_items = data[data['year'] < cutoff_year]['title']

# Önerilerde eski ürünlerin yüzdesi
recommended_items = data['title'].sample(5)  # Örnek öneriler
churn_items = [item for item in recommended_items if item in old_items.values]
freshness = 1 - len(churn_items) / len(recommended_items)
print(f"Freshness: {freshness:.2f}")


Freshness: 0.20


In [88]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, precision_score, recall_score, f1_score
import numpy as np
from collections import defaultdict
import math
from sklearn.metrics import average_precision_score

# Dummy true ratings for users 
true_ratings = {
    'Thermodynamics For Dummies': [5, 4, 3, 4],  # Example for one book: ratings from 4 users
    # Add other books and their ratings here
}

# Dummy predicted ratings for users 
predicted_ratings = {
    'Thermodynamics For Dummies': [4, 5, 3, 4],  # Example for one book: predicted ratings
    # Add other books and their predicted ratings here
}

# Example for MAE, RMSE (you can adjust with your actual ratings)
def compute_mae(true_ratings, predicted_ratings):
    all_true = []
    all_pred = []
    for item in true_ratings:
        all_true.extend(true_ratings[item])
        all_pred.extend(predicted_ratings[item])
    return mean_absolute_error(all_true, all_pred)

def compute_rmse(true_ratings, predicted_ratings):
    all_true = []
    all_pred = []
    for item in true_ratings:
        all_true.extend(true_ratings[item])
        all_pred.extend(predicted_ratings[item])
    return np.sqrt(mean_squared_error(all_true, all_pred))

# Ranking Quality - nDCG, ARHR, RHR
import numpy as np

def compute_ndcg(recommended_items, true_items, user_popularity={'Thermodynamics For Dummies': 100, 'Principles of Thermodynamics': 50}, k=10, epsilon=1e-10):
    if user_popularity is None:
        user_popularity = {}

    # nDCG hesaplama
    dcg = 0
    for rank, item in enumerate(recommended_items[:k], start=1):
        if item in true_items:
            dcg += 1 / np.log2(rank + epsilon)

    # Ideal DCG (IDCG) hesaplama
    idcg = sum(1 / np.log2(i + 1 + epsilon) for i in range(len(true_items)))

    # nDCG sonucu döndür
    return dcg / idcg if idcg != 0 else 0




def compute_arhr(recommended_items, true_items, k=10):
    """
    Compute Average Reciprocal Hit Rank (ARHR).
    """
    arhr = 0.0
    for i in range(min(k, len(recommended_items))):
        if recommended_items[i] in true_items:
            arhr += 1 / (i + 1)
    return arhr / k

def compute_rhr(recommended_items, true_items, k=10):
    """
    Compute Reciprocal Hit Rank (RHR).
    """
    for i in range(min(k, len(recommended_items))):
        if recommended_items[i] in true_items:
            return 1 / (i + 1)
    return 0

# User Experience - Precision, Recall, F1-Score
def compute_precision(true_items, recommended_items, k=10):
    relevant = len(set(true_items) & set(recommended_items[:k]))
    return relevant / k

def compute_recall(true_items, recommended_items, k=10):
    relevant = len(set(true_items) & set(recommended_items[:k]))
    return relevant / len(true_items)

def compute_f1(true_items, recommended_items, k=10):
    precision = compute_precision(true_items, recommended_items, k)
    recall = compute_recall(true_items, recommended_items, k)
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# General Accuracy - Catalog Hit Rate (CHR)
def compute_chr(true_items, recommended_items):
    hits = len(set(true_items) & set(recommended_items))
    return hits / len(true_items)

# Exploration - Serendipity, Novelty, Personalized Novelty
def compute_serendipity(recommended_items, true_items, k=10):
    """
    Serendipity measures how surprising a recommendation is.
    """
    return len([item for item in recommended_items[:k] if item not in true_items]) / k

def compute_novelty(recommended_items, all_items, k=10):
    """
    Novelty measures how unusual or less popular the recommended items are.
    """
    item_popularity = defaultdict(int)
    for item in all_items:
        item_popularity[item] += 1
    return np.mean([1 / np.log2(1 + item_popularity[item]) for item in recommended_items[:k]])

# Diversity - Measures the variability of recommendations
def compute_diversity(recommended_items, k=10):
    """
    Measures diversity by checking item uniqueness.
    """
    return len(set(recommended_items[:k])) / k

# Coverage - Percentage of items that can be recommended
def compute_coverage(all_items, recommended_items):
    """
    Measures how much of the catalog is being covered by recommendations.
    """
    return len(set(recommended_items)) / len(all_items)

# Freshness - Measures the recency of recommended items (using churn)
def compute_freshness(recommended_items, churn_items):
    """
    Measures the freshness by checking how many recent items are recommended.
    """
    return len(set(recommended_items) & set(churn_items)) / len(recommended_items)

# Example: Apply evaluation metrics
recommended_items = ['Thermodynamics For Dummies', 'Thermodynamics: An Engineering Approach with Student Resources DVD', 'Principles of Thermodynamics']  # Replace with actual recommendations
true_items = ['Thermodynamics For Dummies', 'Principles of Thermodynamics']

# For demonstration, you would use real values of true and predicted ratings for each book in the dataset
mae = compute_mae(true_ratings, predicted_ratings)
rmse = compute_rmse(true_ratings, predicted_ratings)

ndcg = compute_ndcg(recommended_items, true_items, k=10)
arhr = compute_arhr(recommended_items, true_items, k=10)
rhr = compute_rhr(recommended_items, true_items, k=10)

precision = compute_precision(true_items, recommended_items, k=10)
recall = compute_recall(true_items, recommended_items, k=10)
f1 = compute_f1(true_items, recommended_items, k=10)

chr = compute_chr(true_items, recommended_items)

serendipity = compute_serendipity(recommended_items, true_items, k=10)
novelty = compute_novelty(recommended_items, data['title'].tolist(), k=10)


diversity = compute_diversity(recommended_items, k=10)
coverage = compute_coverage(data['title'].tolist(), recommended_items)

# Example of churn: Recent items that may have been added
churn_items = ['Thermodynamics For Dummies', 'Fundamentals of Chemical Engineering Thermodynamics']  # Replace with your churn items

freshness = compute_freshness(recommended_items, churn_items)

# Displaying the results
print("Evaluation Metrics:")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"nDCG: {ndcg:.4f}")
print(f"ARHR: {arhr:.4f}")
print(f"RHR: {rhr:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"Catalog Hit Rate (CHR): {chr:.4f}")
print(f"Serendipity: {serendipity:.4f}")
print(f"Novelty: {novelty:.4f}")
print(f"Diversity: {diversity:.4f}")
print(f"Coverage: {coverage:.4f}")
print(f"Freshness: {freshness:.4f}")

Evaluation Metrics:
MAE: 0.5000
RMSE: 0.7071
nDCG: 1.0000
ARHR: 0.1333
RHR: 1.0000
Precision: 0.2000
Recall: 1.0000
F1-Score: 0.3333
Catalog Hit Rate (CHR): 1.0000
Serendipity: 0.1000
Novelty: 1.0000
Diversity: 0.3000
Coverage: 0.0025
Freshness: 0.3333
